In [2]:
#Importamos la librería pandas que necesitamos para la lectura, conversión y limpieza de los datos.
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
import re 

#Importamos librerías necesarias para la visualización.
# -----------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

#Importamos librerías para localizar ruta archivos.
import os
import sys
sys.path.append("../")

# Imputación de nulos usando métodos avanzados estadísticos.
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
# pd.set_option('display.max_rows', None) # para poder visualizar todas las filas de los DataFrames

In [3]:
#Abrimos el csv y lo convertimos a un DataFrame.
df_HR = pd.read_csv('/Users/Tania_1/Desktop/ADALAB/MODULO_3/project-da-promo-angela-modulo-3-team-3/ETL/data/input_data/HR_RAW_DATA.csv', index_col= 0)
df_HR.sample(10)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,employeecount,employeenumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NUMCOMPANIESWORKED,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TOTALWORKINGYEARS,TrainingTimesLastYear,WORKLIFEBALANCE,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YEARSWITHCURRMANAGER,SameAsMonthlyIncome,DateBirth,Salary,RoleDepartament,NUMBERCHILDREN,RemoteWork
107,29,No,travel_rarely,"1389,0$",NaN,21,4,Life Sciences,1,"20,0",2,1,51,4,3,maNufaCturing direcTOr,1,Divorced,NaN,10195,1,Y,No,11,"3,0",3,NaN,1,"10,0",1,"3,0",10,NaN,8,8,NaN,1994,1000000000$,NaN,NaN,0
690,58,No,NaN,"390,0$",Research & Development,1,4,Life Sciences,1,"422,0",4,0,32,1,2,HEalTHCare REPrEsEntATiVE,3,Divorced,NaN,17056,2,NaN,Yes,13,"3,0",4,NaN,1,NaN,2,"3,0",5,NaN,1,2,NaN,1965,1000000000$,HEalTHCare REPrEsEntATiVE - Research & Deve...,NaN,True
841,35,No,travel_frequently,nan$,NaN,-24,3,Medical,1,"269,0",2,1,37,3,2,laborAToRy teCHnIcIAn,2,Single,"4425,0",15986,5,NaN,NaN,11,"3,0",4,NaN,0,NaN,5,"3,0",6,NaN,1,2,"4425,0",1988,1000000000$,NaN,NaN,Yes
973,36,No,travel_rarely,"188,0$",Research & Development,7,4,Other,1,NaN,2,0,65,3,1,ReSEArcH sCIeNtist,4,Single,NaN,23293,2,Y,NaN,18,"3,0",3,"80,0",0,"8,0",6,"3,0",6,NaN,0,1,NaN,1987,1000000000$,ReSEArcH sCIeNtist - Research & Development,NaN,1
759,24,No,NaN,"567,0$",Research & Development,2,1,Technical Degree,1,"1646,0",1,1,32,3,1,ReSeaRch SCIEnTiSt,4,NaN,"3760,0",17218,1,NaN,NaN,13,"3,0",3,NaN,0,"6,0",2,"3,0",6,NaN,1,3,"3760,0",1999,1000000000$,ReSeaRch SCIEnTiSt - Research & Development,NaN,0
1193,29,No,travel_frequently,"1404,0$",NaN,-18,3,Technical Degree,1,NaN,3,1,84,3,1,sALES REprEsEntATive,4,NaN,NaN,18203,1,Y,No,15,"3,0",2,NaN,1,"3,0",5,"3,0",3,NaN,0,2,NaN,1994,1000000000$,NaN,NaN,False
1035,27,No,travel_frequently,"793,0$",NaN,2,1,Life Sciences,1,"1371,0",4,0,43,1,2,sales executiVE,4,Single,"5071,0",20392,3,Y,No,20,"4,0",2,NaN,0,NaN,3,"3,0",6,NaN,0,0,"5071,0",1996,1000000000$,NaN,NaN,False
454,33,No,non-travel,"1283,0$",NaN,2,3,Marketing,1,"1756,0",4,1,62,3,2,sALEs EXEcUtIvE,2,NaN,"5147,0",10697,8,NaN,No,15,"3,0",4,NaN,0,"13,0",2,"2,0",11,NaN,1,7,"5147,0",1990,1000000000$,NaN,NaN,True
595,31,No,travel_rarely,"688,0$",NaN,7,3,NaN,1,NaN,3,0,44,2,3,mAnageR,4,Divorced,NaN,25291,9,Y,NaN,21,"4,0",3,"80,0",1,"10,0",3,"2,0",5,NaN,0,1,NaN,1992,1000000000$,NaN,NaN,Yes
1540,45,No,NaN,"192,0$",NaN,10,2,Life Sciences,1,"544,0",1,0,69,3,1,ResearCH scIeNTiSt,4,Married,"2654,0",9655,3,NaN,No,21,NaN,4,NaN,2,"8,0",3,"2,0",2,NaN,0,2,"2654,0",1978,1000000000$,NaN,NaN,Yes


In [4]:
#FASE 1.
#Creamos la función para leer el archivo csv.
def lectura_archivo(csv):
    df = pd.read_csv(csv, index_col= 0)
    return df

In [ ]:
# lectura_archivo('HR_RAW_DATA.csv')

In [ ]:
#Creamos la función para explorar los datos. 

def exploracion_datos(df):
    print('_____________ INFORMACIÓN GENERAL DEL DATAFRAME ____________\n')
    print(df.info())

    print('___________________ FORMA DEL DATAFRAME ____________________\n')
    
    print(f"El número de filas que tenemos es de {df.shape[0]}.\nEl número de columnas es de {df.shape[1]}\n")
    

    print('_______________ NULOS, ÚNICOS Y DUPLICADOS _________________\n')
    
    print('La cantidad de valores NULOS por columna es de:\n')
    print(df.isnull().sum())
    print('____________________________________________________________\n')

    print('La cantidad de valores ÚNICOS por columna es de:\n')
        
    for columna in df.columns:
        cantidad_valores_unicos = len(df[columna].unique())
    
        print(f'La columna {columna}: {cantidad_valores_unicos}')

    """ Otra forma más rápida de obtener la lista de valores únicos por columna es usando df.nunique()"""

    print('____________________________________________________________\n')

    print('La cantidad de valores DUPLICADOS por columna es de:\n')

    """En análisis posteriores hemos detectado que hay columnas con valores duplicados que nos interesa filtrar, 
    así que vamos a realizar otro bucle for para iterar por todas las columnas del DF y obtener los duplicados de cada una de ellas."""

    for columna in df.columns:
        cantidad_duplicados = df[columna].duplicated().sum()
    
        print(f'La columna {columna}: {cantidad_duplicados}')


    print('____________________ RESUMEN ESTADÍSTICO ____________________')
    print('____________________ Variables Numéricas __________________\n')
    print(df.describe().T)
    
    print('___________________ Variables Categóricas _________________\n')
    print(df.describe(include='object').T)

    


In [ ]:
exploracion_datos(df_HR)

In [ ]:
#FASE 2
""" ¿Qué hacemos con la columna del id de empleados que tiene duplicados? """
df_HR['employeenumber'].duplicated().sum()

In [ ]:
#employeecount: esta columna tiene el mismo valor para todas las filas, por lo que es redundante y se puede eliminar.¿¿??
df_HR['employeecount'].unique()

In [ ]:
#Eliminamos la columna "employeecount"
df_HR.drop(columns='employeecount', inplace= True)

In [ ]:
#NUMBERCHILDREN: Tiene solo valores nulos, se puede eliminar.
df_HR['NUMBERCHILDREN'].isnull().sum()

In [ ]:
#Eliminamos la columna "NUMBERCHILDREN"
df_HR.drop(columns='NUMBERCHILDREN', inplace= True)

In [ ]:
#SameAsMonthlyIncome: parece un duplicado de la columna MonthlyIncome,se puede eliminar. 
df_HR[['SameAsMonthlyIncome', 'MonthlyIncome', 'MonthlyRate', 'HourlyRate']]

In [ ]:
#Eliminamos la columna
df_HR.drop(columns='SameAsMonthlyIncome', inplace= True)
#Buscamos nombre columnas para comprobar que la hemos eliminado.

In [ ]:
#Salary: Tiene un solo valor repetido, por lo que se puede eliminar también.
df_HR['Salary'].unique()

In [ ]:
#Eliminamos la columna "Salary"
df_HR.drop(columns='Salary', inplace= True)

In [ ]:
#Reemplazamos los valores 1 y 0 de la columna Gender por "M" de masculino y "F" de femenino, usando loc.

df_HR.loc[df_HR['Gender'] == 0, 'Gender'] = 'M'
df_HR.loc[df_HR['Gender'] == 1, 'Gender'] = 'F'

print(df_HR['Gender'])


### Función para cambiar a FLOAT

In [ ]:
# Creamos la función 'cambiar comas'

lista_tofloat = ['DailyRate', 'HourlyRate', 'MonthlyIncome']

def cambiar_comas(cadena):
     # Remover el símbolo $, reemplazar comas por puntos
     # Usar expresiones regulares para mantener solo los números y puntos
    try:
    # Remover cualquier caracter que no sea dígito o coma
        cleaned_string = re.sub(r'[^\d,]', '', cadena)
        # Reemplazar comas por puntos
        cleaned_string = cleaned_string.replace(",", ".")
        # Convertir a float
        return float(cleaned_string)
    except:
        return np.nan # Devuelve un np.nan si hay un valor inválido

# Aplicar la función a cada columna en la lista
for col in lista_tofloat:
    df_HR[col] = df_HR[col].apply(cambiar_comas)


# Comprobamos que la columna, que antes era de tipo string, ahora es float64.
df_HR[lista_tofloat].dtypes



In [ ]:
df_HR.head(10)

In [ ]:
#lo guardo para que se guarden los datos

df_HR.to_csv("df_HR_limpio.csv", index = False)

### Función para cambiar a INT

In [ ]:
"""# Creamos la función 'cambiar comas'

lista_toint = ['WORKLIFEBALANCE']

def cambiar_comas(cadena):
     # Remover el símbolo $, reemplazar comas por puntos
     # Usar expresiones regulares para mantener solo los números y puntos
    try:
    # Remover cualquier caracter que no sea dígito o coma
        cleaned_string = re.sub(r'[^\d,]', '', cadena)
        # Reemplazar comas por puntos
        cleaned_string = cleaned_string.replace(",", ".")
        # Convertir a float
        return int(cleaned_string)
    except:
        return np.nan # Devuelve un np.nan si hay un valor inválido

# Aplicar la función a cada columna en la lista
for col in lista_toint:
    #convertir a int
    df_HR[col] = df_HR[col].apply(cambiar_comas).astype('Int64')


# Comprobamos que la columna, que antes era de tipo string, ahora es float64.
df_HR[lista_toint].dtypes"""

In [ ]:
"""def comasapuntos(num):
    try:
        return float(num.replace(",", "."))
    except:
        return np.nan

columnas = ["WORKLIFEBALANCE"]

for col in columnas:
    df_HR[col] = df_HR[col].apply(comasapuntos)
"""

In [ ]:
df_HR['WORKLIFEBALANCE'] = df_HR['WORKLIFEBALANCE'].replace('NaN', pd.NA)

# Convertir los valores en 'WORKLIFEBALANCE' a float y manejar los NaN
df_HR['WORKLIFEBALANCE'] = df_HR['WORKLIFEBALANCE'].astype(str).str.replace(',', '.').astype(float)

# Convertir los valores a int, ignorando los NaN
df_HR['WORKLIFEBALANCE'] = df_HR['WORKLIFEBALANCE'].astype('Int64')
df_HR['WORKLIFEBALANCE']

In [ ]:
#pasar a csv para que se guarde

df_HR.to_csv('df_HR_limpio.csv')

In [ ]:
df_HR = pd.read_csv('df_HR_limpio.csv')
df_HR.sample(10)

In [ ]:
#ver los datos unicos

df_HR['Age'].unique()

In [ ]:
#Cambiamos el tipo de dato de la columna Age para que sea interger.

df_HR.loc[df_HR['Age'] == 'forty-seven', 'Age'] = '47'
df_HR.loc[df_HR['Age'] == 'fifty-eight', 'Age'] = '58'
df_HR.loc[df_HR['Age'] == 'thirty-six', 'Age'] = '36'
df_HR.loc[df_HR['Age'] == 'fifty-five', 'Age'] = '55'
df_HR.loc[df_HR['Age'] == 'fifty-two', 'Age'] = '52'
df_HR.loc[df_HR['Age'] == 'thirty-one', 'Age'] = '31'
df_HR.loc[df_HR['Age'] == 'thirty', 'Age'] = '30'
df_HR.loc[df_HR['Age'] == 'twenty-six', 'Age'] = '26'
df_HR.loc[df_HR['Age'] == 'thirty-seven', 'Age'] = '37'
df_HR.loc[df_HR['Age'] == 'thirty-two', 'Age'] = '32'
df_HR.loc[df_HR['Age'] == 'thirty-seven', 'Age'] = '37'
df_HR.loc[df_HR['Age'] == 'twenty-four', 'Age'] = '24'

df_HR['Age']= df_HR['Age'].astype(int)
df_HR['Age'].dtype

In [ ]:
#Buscamos los valores negativos de la columna 'DistanceFromHome'
valores_negativos = df_HR[df_HR['DistanceFromHome'] < 0]['DistanceFromHome']
print(valores_negativos)
print('_________________________________________________')
print(f'Hay {valores_negativos.shape[0]} filas con valores negativos')

In [ ]:
#Sustituimos valores negativos por positivos
df_HR.loc[df_HR['DistanceFromHome'] < 0, 'DistanceFromHome'] = df_HR['DistanceFromHome'].abs()

#Comprobamos que ya no hay valores negativos
df_HR[df_HR['DistanceFromHome'] < 0]

In [ ]:
#Corregimos errores tipográficos en la columna MaritalStatus.
df_HR['MaritalStatus'].unique()

In [ ]:
#Como 'Marreid'.
df_HR.loc[df_HR['MaritalStatus'] == 'Marreid', 'MaritalStatus'] = 'Married'
df_HR['MaritalStatus'].unique()

#O 'divorced'.
df_HR.loc[df_HR['MaritalStatus'] == 'divorced', 'MaritalStatus'] = 'Divorced'
df_HR['MaritalStatus'].unique()

In [ ]:
""" Mirar qué otras columnas podemos cambiar de typo porque no estén en el correcto, como NUMBERCHILDRENS.
Ver si alguna otra columna tiene muchos nulos y no nos interesa (de cara a eliminarla). """

In [ ]:
#FASE 4
#Seudocódigo.
""" Usar como filtro los valores de la columna **Attrition**. Los valores == YES han abandonado la empresa. 
    Buscar su nivel de Satisfacción. Columnas Age, Gender, EnviromentSatisfaction, JobInvolvement, 
    JobSatisfaction, 'RelationshipSatisfaction, WORKLIFEBALANCE, YearsSinceLastPromotion, ¿ YEARSWITHCURRMANAGER ?, 
    Salary, RoleDepartament, NUMBERCHILDREN, DistanceFromHome - RemoteWork. """

In [ ]:
df_HR.to_csv('df_HR_limpio.csv', index=False)

In [ ]:
#ABRO el nuevo archivo, el limpio para seguir trabajando

df_limpio = pd.read_csv('df_HR_limpio.csv', index_col= 0)
df_limpio.head(2)

In [ ]:
# Función para mapear los valores a 'yes' y 'no', RemoteWork
def map_values(value):
    if str(value).lower() in ['1', 'true', 'yes']:
        return 'yes'
    elif str(value).lower() in ['0', 'false', 'no']:
        return 'no'
    else:
        return value  

# Aplicar la función a la columna
df_limpio['RemoteWork'] = df_limpio['RemoteWork'].apply(map_values)



In [ ]:
df_limpio.sample(5)

In [ ]:
df_limpio.to_csv('df_HR_limpio.csv', index = False)

In [ ]:
df_HR = pd.read_csv('df_HR_limpio.csv', index_col= False)

### Corregir mayusculas y minusculas

In [ ]:
df_HR["JobRole"] = df_HR["JobRole"].str.title()
df_HR["RoleDepartament"] = df_HR["RoleDepartament"].str.title()
df_HR.rename(columns={'TOTALWORKINGYEARS': 'TotalWorkingYears'}, inplace=True)
df_HR.rename(columns={'employeenumber': 'EmployeeNumber'}, inplace=True)
df_HR.rename(columns={'NUMCOMPANIESWORKED': 'NumCompaniesWorked'}, inplace=True)
df_HR.rename(columns={'WORKLIFEBALANCE': 'WorkLifeBalance'}, inplace=True)
df_HR.rename(columns={'YEARSWITHCURRMANAGER': 'YearsWithCurrManager'}, inplace=True)

In [ ]:
df_HR.sample(10)

In [ ]:
df_HR.to_csv("df_HR_limpio.csv", index = False)

In [ ]:
# Eliminamos tres columna más
lista_eliminar = ["Over18", "StandardHours", "RoleDepartament", "Department", "YearsInCurrentRole"]
df_HR.drop(columns= lista_eliminar, inplace= True)

In [ ]:
df_HR.columns

In [ ]:
# FUNCIÓN DE CONTEO DE FRECUENCIAS DE CADA COLUMNA

# EJECUTAR Y ABRIR EN TEXT EDITOR PARA VISUALIZAR LOS RESULTADOS EN UN ARCHIVO INDEPENDIENTE

def count_frequencies(df):
    # Creamos un diccionario para almacenar los resultados
    frequency_counts = {}
    
    # Recorremos todas las columnas del DataFrame
    for column in df_HR.columns:
        # Aplicar .value_counts() a cada columna y almacenarlo en el diccionario
        frequency_counts[column] = df[column].value_counts()
    
    return frequency_counts

# Llamamos a la función y obtenemos el conteo de frecuencia de cada columna
frequency_counts = count_frequencies(df_HR)

# Mostramos los resultados
for column, counts in frequency_counts.items():
    print(f"Conteo de frecuencias para la columna {column}:")
    print(counts)
    print()  # Línea en blanco para separar los resultados

In [ ]:
# Eliminamos los duplicados de la columna EmployeeNumber
df_HR = df_HR.drop_duplicates(subset=['EmployeeNumber'])

In [ ]:
# Comprobamos que se han eliminado los nulos
df_HR["EmployeeNumber"].duplicated().sum()

In [ ]:
df_HR.shape

In [ ]:
""" Para obtener los duplicados de todas las columnas

for columna in df_HR:
    num_filas_duplicadas = df_HR[columna].duplicated().sum()
    print(f'{columna.upper()}: {num_filas_duplicadas}')"""

In [ ]:
# Para obtener los nulos de todas las columnas

for columna in df_HR:
    num_filas_nulas = df_HR[columna].isnull().sum()
    print(f'{columna.upper()}: {num_filas_nulas}')

## Gestión de nulos de columnas categóricas

In [ ]:
# Calculamos el porcentaje de nulos que tenemos para cada columna categórica
porc_nulos = (df_HR.isnull().sum() / df_HR.shape[0]) * 100
porc_nulos

# Lo convertimos a DataFrame
df_nulos = pd.DataFrame(porc_nulos, columns = ["%_nulos"])

# Filtramos el DataFrame para quedarnos solo con aquellas columnas que tengan nulos
df_nulos[df_nulos["%_nulos"] > 0]

In [ ]:
# Obtenemos la lista de columnas categóricas que tienen nulos
nulos_esta_cat = df_HR[df_HR.columns[df_HR.isnull().any()]].select_dtypes(include = "O").columns
print("Las columnas categóricas que tienen nulos son : \n ")
print(nulos_esta_cat)

In [ ]:
# Sacamos el 'value_counts()' de cada una de las columnas categóricas que tienen nulos para saber como es la distribución de sus categorías
for col in nulos_esta_cat:
    print(f"La distribución de las categorías para la columna {col.upper()}")
    display(df_HR[col].value_counts() / df_HR.shape[0] * 100)  # display es una función utilizada para mostrar objetos de manera más legible en Jupyter Notebooks o entornos similares. 
    print("........................")

In [ ]:
# Sustituimos los valores nulos de las siguientes columnas por "Unkown", ya que no hay mayorías claras que nos permitan imputar.

columnas_desconocido = ['BusinessTravel', 'EducationField', 'EmployeeNumber',
       'MaritalStatus', 'OverTime', 'PerformanceRating', 'TotalWorkingYears']
    
    
for columna in columnas_desconocido:
    
    # reemplazamos los nulos por el valor Unknown para cada una de las columnas de la lista
    df_HR[columna] = df_HR[columna].fillna("Unknown")
    
# comprobamos si quedan nulos en las columnas categóricas. 
print("Después del reemplazo usando 'fillna' quedan los siguientes nulos")
df_HR[columnas_desconocido].isnull().sum()

In [ ]:
df_HR.columns

In [ ]:
df_HR.head()    

In [ ]:
df_HR.dtypes

## Gestión de nulos de columnas numéricas

In [ ]:
# Obtenemos la lista de columnas numericas que tienen nulos
nulos_esta_num = df_HR[df_HR.columns[df_HR.isnull().any()]].select_dtypes(include = np.number).columns
print("Las columnas numéricas que tienen nulos son : \n ")
print(nulos_esta_num)

In [ ]:
# Calculamos el porcentaje de nulos que tenemos en cada una de las columnas numericas
df_HR[nulos_esta_num].isnull().sum() / df_HR.shape[0] *100

In [ ]:
# Hacemos un Boxplot para ver la distribución de los datos
plt.figure(figsize=(5, 2.5))
sns.boxplot(x=df_HR['DailyRate'])
plt.title('Boxplot DailyRate')
plt.xlabel('Valor')
plt.show()

In [ ]:
# Como la distribución de los datos es aproximadamente simétrica (los datos se distribuyen uniformemente alrededor del centro), la media es una buena medida central
df_HR['DailyRate'].fillna(df_HR['DailyRate'].mean(), inplace=True)

In [ ]:
# Hacemos un Boxplot para ver la distribución de los datos
plt.figure(figsize=(5, 2.5))
sns.boxplot(x=df_HR['HourlyRate'])
plt.title('Boxplot HourlyRate')
plt.xlabel('Valor')
plt.show()

In [ ]:
# Como la distribución de los datos es aproximadamente simétrica (los datos se distribuyen uniformemente alrededor del centro), la media es una buena medida central
df_HR['HourlyRate'].fillna(df_HR['HourlyRate'].mean(), inplace=True)

In [ ]:
# Hacemos un Boxplot para ver la distribución de los datos
plt.figure(figsize=(5, 2.5))
sns.boxplot(x=df_HR['MonthlyIncome'])
plt.title('Boxplot MonthlyIncome')
plt.xlabel('Valor')
plt.show()

In [ ]:
"""# Como la distribución de los datos es asimétrica, imputamos con la mediana
df_HR['MonthlyIncome'].fillna(df_HR['MonthlyIncome'].median(), inplace=True)"""

# Como el porcentaje de nulos es tan alto usaremos el método IterativeImputer, algo más preciso que la media y la meediana
# Instanciamos IterativeImputer
imputer_iterative = IterativeImputer(max_iter=20, random_state=42)

# Ajustamos y transformamos los datos
imputer_iterative_imputado = imputer_iterative.fit_transform(df_HR[["MonthlyIncome"]])

# Comprobamos el array imputado
print("\nArray imputado:")
print(imputer_iterative_imputado)

# Convertimos el resultado de vuelta a un DataFrame
df_HR["MonthlyIncome"] = imputer_iterative_imputado

In [ ]:
# Hacemos un Boxplot para ver la distribución de los datos
plt.figure(figsize=(5, 2.5))
sns.boxplot(x=df_HR['WorkLifeBalance'])
plt.title('Boxplot WorkLifeBalance')
plt.xlabel('Valor')
plt.show()

In [ ]:
# Como la distribución de los datos es aproximadamente simétrica (los datos se distribuyen uniformemente alrededor del centro), la media es una buena medida central
df_HR['WorkLifeBalance'].fillna(df_HR['WorkLifeBalance'].mean(), inplace=True)

In [ ]:
# Comprobamos que ahora ya no hay valores nulos en esas columnas
nulos_esta_num = df_HR[df_HR.columns[df_HR.isnull().any()]].select_dtypes(include = np.number).columns
print("Las columnas numéricas que tienen nulos son : \n ")
print(nulos_esta_num)

In [ ]:
df_HR.to_csv('df_HR_limpio.csv', index=False)

### Crear los csv para importarlos como tablas en sql

In [ ]:
#definimos las columnas que queremos en cada variable 

employee = ['Age', 'Gender', 'MaritalStatus', 'DateBirth', 'Education', 'EducationField', 'Attrition', 'BusinessTravel', 'DistanceFromHome' ]
job_details = ['JobRole','JobLevel', 'NumCompaniesWorked', 'TotalWorkingYears', 'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager']
compensation = ['DailyRate','HourlyRate','MonthlyIncome','MonthlyRate','StockOptionLevel','PercentSalaryHike','OverTime','TrainingTimesLastYear','RemoteWork']
satisfaction = ['EnvironmentSatisfaction','JobInvolvement','JobSatisfaction','RelationshipSatisfaction','WorkLifeBalance','PerformanceRating']


In [ ]:
# Cargar el CSV y seleccionar solo las columnas deseadas
df = pd.read_csv('df_HR_limpio.csv', usecols=employee)
df1 = pd.read_csv('df_HR_limpio.csv', usecols=job_details)
df2 = pd.read_csv('df_HR_limpio.csv', usecols=compensation)
df3 = pd.read_csv('df_HR_limpio.csv', usecols=satisfaction)

In [ ]:
df.to_csv('Employees.csv', index=False)
df1.to_csv('JobDetails.csv', index=False)
df2.to_csv('Compensation.csv', index=False)
df3.to_csv('Satisfaction.csv', index = False)

In [ ]:
remote_work = pd.read_csv('Compensation.csv')
remote_work.sample(5)